In [5]:
import psycopg2 as ps
import pandas as pd
import random as rm
from copy import copy
import glob

Download data from maalfrid_prod

In [2]:
conn = ps.connect(dbname="maalfrid_prod", user="larsm", password="sprakbank", host="localhost", port="5434")

def query(q, conn = conn):
    return pd.read_sql(q, conn)

In [3]:
# Get 100 random id's from each scraping
# maal1 = query("select * from paths_maalfrid_1 ORDER BY random() LIMIT 100")
# maal2 = query("select * from paths_maalfrid_3 ORDER BY random() LIMIT 100")
# maal1.to_csv("data/maalfrid_1_sample.csv")
# maal2.to_csv("data/maalfrid_2_sample.csv")

In [6]:
dct = {}
names = ["maal1", "maal2"]

for name, path in zip(names, sorted(glob.glob("data/*"))):
    dct[name] = {"ids" : pd.read_csv(path, index_col=0) }

In [8]:
for i in dct.keys():
    lst = ", ".join([str(x) for x in dct[i]["ids"].fulltext_id.values])
    df = query("SELECT * FROM fulltext WHERE fulltext_id in ({})".format(lst))
    dct[i].update({"texts" : df})
    df.to_csv(f"data/{i}_fulltext.csv")

In [10]:
for i in dct.keys():
    lst = ", ".join([str(x) for x in dct[i]["ids"].fulltext_id.values])
    df = query("SELECT * FROM simhash WHERE fulltext_id in ({})".format(lst))
    dct[i].update({"simhash" : df})
    df.to_csv(f"data/{i}_simhash.csv")